# Introduction to trackintel

This tutorial will show you the most important functionalities of trackintel.

## Model

Trackintel is based on a hierachical data model. 
![title](../docs/assets/hierarchy.png)

The details are explained in [readme.md](https://github.com/mie-lab/trackintel/blob/master/README.md)

To get started, import the framework using the following command. In addition it is always useful to import also pandas and geopandas.

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import trackintel as ti
import geopandas as gpd
import pandas as pd
import numpy as np
import os

As a first step, the data that should be processed has to be importet. In this example we use a stored GEOJSON file, that contains raw positionfixes.

In [ ]:
gdf = gpd.read_file('./data/pfs_tutorial.geojson')

This is the reccomended way to import data. Nevertheless it is also possible to read csv files directly using [read_positionfixes_csv](https://trackintel.readthedocs.io/en/latest/modules/io.html#csv-file-import).

The GeoDataFrame (gdf) imported by the GeoPandas read_file function is not yet in the correct format to be used by trackintel. If you try to call a trackintel function (e.g. plotting) on this GeoDataFrame, an error will occur. 

In [ ]:
#gdf.as_positionfixes.plot()

gdf

To transform the GeoDataFrame into the correct format use the function [positionfixes_from_gpd](https://trackintel.readthedocs.io/en/latest/modules/io.html#geodataframe-import). The method allows to rename the columns, define an index column and define a timezone. If no timezone is defined, UTC is assumed.

In [ ]:
pfs = ti.io.from_geopandas.read_positionfixes_gpd(gdf, tz='UTC', 
                                                  mapper={'User':'user_id','time':'tracked_at', 'geometry':'geom'})
pfs.as_positionfixes.plot()

When the GeoDataFrame is once in the right format, all trackintel functions can be called using the accessor _as_positionfixes_ (as_triplegs, as_staypoints, ... for the other data types). In the next steps the staypoints and triplegs are generated.

In [ ]:
pfs, stps = pfs.as_positionfixes.generate_staypoints(method='sliding', dist_threshold=25, time_threshold=5*60)

Each data type in the model has a plotting function. Let's vizualize the generated staypoints. To get a background map for bigger data sets, use plot_osm=True.

In [ ]:
stps.as_staypoints.plot(positionfixes=pfs, radius=20)

In [ ]:
pfs, tpls = pfs.as_positionfixes.generate_triplegs(stps, method='between_staypoints')

The generated staypoints and triplegs can be exported to csv files. Once stored, they can be re imported for further treatment.

In [ ]:
tpls.as_triplegs.plot(positionfixes=pfs,staypoints=stps, staypoints_radius=10)


The generated data can be exported as csv files and reused for further processing directly in trackintel or any other framework. Within python it is probably the easiest way just to keep the GeoDataFrames.

In [ ]:
pfs.as_positionfixes.to_csv('./out/pfs_tutorial.csv')
stps.as_staypoints.to_csv('./out/stps_tutorial.csv')
tpls.as_triplegs.to_csv('./out/tpls_tutorial.csv')

To continue the analysis in a new script, the exported triplegs (or staypoints, positionfixes, ...) can be reimported directly from the csv file. This is done here for demonstration purposes. When reading csv files, it is important to specify the coordinate reference system (crs) with the EPSG code.

In [ ]:
tpls = ti.read_triplegs_csv('./out/tpls_tutorial.csv', index_col=None, crs=4326)

A basic analysis functionality provided by trackintel is the [transport mode prediction](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#transport-mode-identification). Per default it offers a split in to slow, motorized and fast mobility.

In [ ]:
tpls.as_triplegs.predict_transport_mode()

The speed categories can also be defined individually by specifying the upper border of the interval corresponding to the category. For example:

In [ ]:
tpls.as_triplegs.predict_transport_mode(categories={5/3.6:'cat1',10/3.6:'cat2',20/3.6:'cat3',np.inf:'cat4'})

The data set can also be analized in terms of temporal coverage/tracking quality. The temporal [tracking quality](https://trackintel.readthedocs.io/en/latest/modules/analysis.html#tracking-quality) can be shown per user in different granularities.

In [ ]:
ti.temporal_tracking_quality(tpls, granularity='all')

Further functions for more detailed analysis and higher aggregation levels (trips, locations, ...) can be found in the [trackintel documentation](https://trackintel.readthedocs.io/en/latest/). 

Thanks for using trackintel. If you discover any bug, have an idea for a usful funcionality feel free to open an [issue](https://github.com/mie-lab/trackintel/issues)